In [1]:

# import required packages
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import LeakyReLU

# Initialize image data generator with rescaling
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)

# Preprocess all test images
train_generator = train_data_gen.flow_from_directory(
        '/kaggle/input/emotion-detection-fer/train',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

# Preprocess all train images
validation_generator = validation_data_gen.flow_from_directory(
        '/kaggle/input/emotion-detection-fer/test',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

# create model structure
emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation=LeakyReLU(alpha=0.1), input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation=LeakyReLU(alpha=0.1)))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation=LeakyReLU(alpha=0.1)))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation=LeakyReLU(alpha=0.1)))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation=LeakyReLU(alpha=0.1)))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

cv2.ocl.setUseOpenCL(False)

emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001, decay=1e-6), metrics=['accuracy'])

# Train the neural network/model
emotion_model_info = emotion_model.fit_generator(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=100,
        validation_data=validation_generator,
        validation_steps=7178 // 64)

# save model structure in jason file
model_json = emotion_model.to_json()
with open("emotion_model.json", "w") as json_file:
    json_file.write(model_json)

# save trained model weight in .h5 file
emotion_model.save_weights('emotion_model.h5')



Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
448/448 [==============================] - 155s 332ms/step - loss: 1.8243 - accuracy: 0.2492 - val_loss: 1.6611 - val_accuracy: 0.3523
Epoch 2/100
448/448 [==============================] - 38s 85ms/step - loss: 1.6426 - accuracy: 0.3617 - val_loss: 1.5382 - val_accuracy: 0.4201
Epoch 3/100
448/448 [==============================] - 33s 75ms/step - loss: 1.5449 - accuracy: 0.4087 - val_loss: 1.4593 - val_accuracy: 0.4445
Epoch 4/100
448/448 [==============================] - 33s 73ms/step - loss: 1.4644 - accuracy: 0.4415 - val_loss: 1.4025 - val_accuracy: 0.4621
Epoch 5/100
448/448 [==============================] - 33s 73ms/step - loss: 1.4019 - accuracy: 0.4709 - val_loss: 1.3481 - val_accuracy: 0.4904
Epoch 6/100
448/448 [==============================] - 32s 71ms/step - loss: 1.3372 - accuracy: 0.4946 - val_loss: 1.3124 - val_accuracy: 0.4957
Epoch 7/100
448/448 [==============================] - 32s 71ms/step - loss: 1.2919 - accuracy: 0.5166 - val_loss: 1.2649 - val_

In [3]:
import cv2
import numpy as np
from keras.models import model_from_json
from keras.layers import LeakyReLU

# Define the emotion dictionary
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

# Load the pre-trained model from JSON and weights
json_file = open('/kaggle/working/emotion_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()


# Load the pre-trained model from JSON and weights with custom_objects
emotion_model = model_from_json(loaded_model_json, custom_objects={'LeakyReLU': LeakyReLU})
emotion_model.load_weights("/kaggle/working/emotion_model.h5")
print("Loaded model from disk")


# Initialize face detector
face_detector = cv2.CascadeClassifier('/kaggle/input/default/haarcascade_frontalface_default.xml')

# Start the video capture (change video path as needed)
cap = cv2.VideoCapture("/kaggle/input/sample-video/WIN_20230205_21_34_30_Pro.mp4")

# Define the codec and create VideoWriter object
output_path = '/kaggle/working/output_video_with_emotions.avi'
codec = cv2.VideoWriter_fourcc(*'XVID')
output_video = cv2.VideoWriter(output_path, codec, 30.0, (1280, 720))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize frame if needed
    frame = cv2.resize(frame, (1280, 720))

    # Convert frame to grayscale for face detection
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    num_faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

    # Process each detected face
    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (0, 255, 0), 4)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)

        # Predict emotions for the face
        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

    # Write the frame with annotations to the output video
    output_video.write(frame)

cap.release()
output_video.release()

# Print the path to the output video
print(f"Output video saved at: {output_path}")


Loaded model from disk
Output video saved at: /kaggle/working/output_video_with_emotions.avi
